# Point Regressor Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from pointRegressorSurrogateModel import PointRegressor

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train and test random forest point regressors

In [5]:
rf = PointRegressor('Random Forest')
rf.trainModel(trainData, valData, 
             flatten=False, 
             logTrans=False, 
             ssTrans=True, 
             saveDir='./results/rf01/')

trainRes = rf.testModel(trainData)
testRes = rf.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

trained 30 random forest models in 14.14 seconds


,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000004,0.001045,0.043597,0.956961,0.998317,0.981528,0.961902
test,0.000021,0.002503,0.101589,0.677624,0.997297,0.878046,0.752604


## 4. Visualize some predictions

In [8]:
i = 8
pred = rf.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [7]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = rf.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)